In [1]:
# STEP — Call /game/{gamePk}/boxscore for your chosen game

import requests
from pprint import pprint

game_pk = 776458
box_url = f"https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore"

response = requests.get(box_url)
print("HTTP status:", response.status_code)
print("Requested URL:", response.url)

box = response.json()

# STEP — Inspect the structure of the /game/{gamePk}/boxscore response

# 1. Look at the top-level dictionary keys
#    This shows the big "sections" included in the boxscore JSON
print("Top-level keys in boxscore response:")
print(list(box.keys()))   # e.g. ['teams', 'officialScorer', 'info', 'pitchingNotes']

# 2. Look inside the "teams" section
#    This tells you which sub-sections exist under 'teams' (usually 'home' and 'away')
teams_section = box.get("teams", {})
print("\nKeys inside 'teams':")
print(list(teams_section.keys()))   # should print ['home', 'away']



HTTP status: 200
Requested URL: https://statsapi.mlb.com/api/v1/game/776458/boxscore
Top-level keys: ['copyright', 'teams', 'officials', 'info', 'pitchingNotes', 'topPerformers']
Teams keys: ['away', 'home']


In [5]:
# STEP — Build Boxscore Silver (player-level DataFrame)

import pandas as pd

def extract_players(side_name: str, side_dict: dict, game_pk: int) -> list[dict]:
    """
    Extract per-player rows from one side ('home' or 'away') of a boxscore.
    """
    rows = []
    team_info = side_dict.get("team", {}) or {}
    players = side_dict.get("players", {}) or {}

    for pid, pdata in players.items():
        person = pdata.get("person", {}) or {}
        pos    = pdata.get("position", {}) or {}
        stats  = pdata.get("stats", {}) or {}

        batting  = stats.get("batting", {}) or {}
        pitching = stats.get("pitching", {}) or {}
        fielding = stats.get("fielding", {}) or {}

        row = {
            # Context
            "gamePk": game_pk,
            "team_side": side_name,                         # 'home' or 'away'
            "team_id": team_info.get("id"),
            "team_name": team_info.get("name"),

            # Player identity
            "player_id": person.get("id"),
            "player_fullName": person.get("fullName"),
            "position": pos.get("abbreviation"),

            # Batting
            "bat_AB": batting.get("atBats"),
            "bat_R": batting.get("runs"),
            "bat_H": batting.get("hits"),
            "bat_2B": batting.get("doubles"),
            "bat_3B": batting.get("triples"),
            "bat_HR": batting.get("homeRuns"),
            "bat_RBI": batting.get("rbi"),
            "bat_BB": batting.get("baseOnBalls"),
            "bat_SO": batting.get("strikeOuts"),
            "bat_SB": batting.get("stolenBases"),
            "bat_CS": batting.get("caughtStealing"),
            "bat_AVG": batting.get("avg"),
            "bat_OBP": batting.get("obp"),
            "bat_SLG": batting.get("slg"),
            "bat_OPS": batting.get("ops"),

            # Pitching
            "pit_IP": pitching.get("inningsPitched"),
            "pit_H": pitching.get("hits"),
            "pit_R": pitching.get("runs"),
            "pit_ER": pitching.get("earnedRuns"),
            "pit_BB": pitching.get("baseOnBalls"),
            "pit_SO": pitching.get("strikeOuts"),
            "pit_HR": pitching.get("homeRuns"),
            "pit_PC": pitching.get("pitchesThrown"),
            "pit_STR": pitching.get("strikes"),
            "pit_ERA": pitching.get("era"),
            "pit_WHIP": pitching.get("whip"),

            # Fielding (optional — included here for completeness)
            "fld_PO": fielding.get("putOuts"),
            "fld_A": fielding.get("assists"),
            "fld_E": fielding.get("errors"),
            "fld_TC": fielding.get("chances"),
        }
        rows.append(row)
    return rows

# Extract players for both sides
rows_home = extract_players("home", box["teams"]["home"], game_pk)
rows_away = extract_players("away", box["teams"]["away"], game_pk)

# Combine into one DataFrame
df_box_players = pd.DataFrame(rows_home + rows_away)

print("Shape:", df_box_players.shape)
df_box_players.head(10)


Shape: (56, 37)


,gamePk,team_side,team_id,team_name,player_id,player_fullName,position,bat_AB,bat_R,bat_H,...,pit_SO,pit_HR,pit_PC,pit_STR,pit_ERA,pit_WHIP,fld_PO,fld_A,fld_E,fld_TC
0,776458,home,139,Tampa Bay Rays,664040,Brandon Lowe,2B,4.0,0.0,1.0,...,NaN,NaN,NaN,NaN,None,None,1.0,1.0,0.0,2.0
1,776458,home,139,Tampa Bay Rays,666624,Christopher Morel,DH,4.0,2.0,3.0,...,NaN,NaN,NaN,NaN,None,None,0.0,0.0,0.0,0.0
2,776458,home,139,Tampa Bay Rays,664126,Pete Fairbanks,P,NaN,NaN,NaN,...,1.0,2.0,21.0,15.0,None,None,0.0,1.0,0.0,1.0
3,776458,home,139,Tampa Bay Rays,686752,Ryan Pepiot,P,NaN,NaN,NaN,...,6.0,0.0,90.0,57.0,None,None,0.0,0.0,0.0,0.0
4,776458,home,139,Tampa Bay Rays,700246,Carson Williams,SS,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,None,None,0.0,1.0,0.0,1.0
5,776458,home,139,Tampa Bay Rays,680700,Richie Palacios,LF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
6,776458,home,139,Tampa Bay Rays,687003,Brian Van Belle,P,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
7,776458,home,139,Tampa Bay Rays,669438,Mason Englert,P,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
8,776458,home,139,Tampa Bay Rays,669358,Shane Baz,P,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
9,776458,home,139,Tampa Bay Rays,802415,Chandler Simpson,LF,4.0,0.0,0.0,...,NaN,NaN,NaN,NaN,None,None,3.0,0.0,0.0,3.0
